# 5.A. What is a recurrence matrix?

Most dynamical systems of interest evolve within a **bounded set**. This means that all trajectories move around in a specifc region in the phase space instead of escaping to infinity.

In such cases a **recurrence** is an event likely to happen (and most of the time it is guaranteed to happen). Recurrence simply means that a trajectory $x(t_i)$ "visits the neighborhood" of itself at a another point in time, $x(t_j)$. A **Recurrence Matrix** is a way to list these recurrences.

To put it into more concrete terms, the recurrence matrix is defined as

$$
R[i, j] = \begin{cases}
1 \quad \text{if}\quad d(x[i], x[j]) \le \varepsilon\\
0 \quad \text{else}
\end{cases}
$$

with $d(x[i], x[j])$ the distance between the two points at times $i, j$. You can use different metrics for $d$ and by default we use the Euclidean.

Notice that $x$ can be single timeseries or a full trajectory. 

---

In the previous examples we have been looking at the Lorenz system in different regimes. For starters lets see how the recurrence matrix the Lorenz system looks like in these cases

In [ ]:
using DynamicalSystems, Plots

In [ ]:
lor = Systems.lorenz()

To create a recurrence matrix simply use `RecurrenceMatrix(trajectory, ε)` with `ε` the distance threshold.

In [ ]:
set_parameter!(lor, 2, 69.75) # stable regime for Lorenz

t = 5.0
dt = 0.01

tr = trajectory(lor, t; dt = dt, Ttr = 2000.0)

R = RecurrenceMatrix(tr, 5.0)

## 5.B. Recurrence plots

Internally a recurrence matrix is represented as a sparse matrix. 
What you see above is a simple ASCII-based plot of the matrix (with dots at non-zero elements). 

There are three ways to visualize a recurrence matrix:

1. Using `recurrenceplot` for a text-based plot (for use in e.g. a terminal). This is the only function that "plots" immediatelly
2. Using `coordinates` which returns the coordinates of the nonzero elements as scatter data.
3. Using `grayscale` which creates a grayscale matrix representation.

In [ ]:
x, y = coordinates(R)
hcat(x, y)

To plot this one has to use the plotting library of their choice:

In [ ]:
scatter(x, y, 
    markersize=0.2, alpha = 0.5, xlims=(1, size(R)[1]), ylims=(1, size(R)[2]),
    size=(500,500),leg=false, grid=false)

Notice that the above plot has its axis in index coordinates. One can modify this by indexing the time vector of the trajectory, which by default is `0:dt:t`.

In [ ]:
tvec = 0:dt:t

scatter(tvec[x], tvec[y], 
    markersize=0.2, alpha = 0.5, xlims=(0,t), ylims=(0,t),
    size=(500,500),leg=false, grid=false)

## 5.C. Typical recurrence plots

In the previous notebooks we explored the Lorenz system in a regular and chaotic regime.
Here is a comparison of a recurrence plot of the Lorenz system in these two regimes:

In [ ]:
lor = Systems.lorenz()
subplots = []

for (i, ρ) in enumerate((69.75, 28.0))
    set_parameter!(lor, 2, ρ)
    t, dt = 20.0, 0.02
    tr = trajectory(lor, t; dt = dt, Ttr = 2000.0)
    tvec = 0:dt:t
    
    p = plot(tr[:, 1], tr[:, 3], color =:black, label = "X vs Z",
             title=string("\\rho = $ρ, ", (i != 1 ? "not periodic" : "periodic")))
    push!(subplots, p)
    
    ε = (i == 1 ? 5.0 : 3.0)
    R = RecurrenceMatrix(tr, ε)
    
    x, y = coordinates(R)
    q = scatter(tvec[x], tvec[y], markersize=0.3, legend=false, alpha = 0.2, color=:black, xlims=(0,t), ylims=(0,t), xlabel="t")
    if i == 1 
        plot!(ylabel="t")
    end
    push!(subplots, q)
end

permute!(subplots, [1,3,2,4])
plot(subplots..., layout=(2,2), size=(1100, 1000), html_output_format=:png, grid=false)

On the left we see long (potentially infinite) diagonals repeated for different times. This is the case for periodic systems as they visit exactly the same area on the phase space again and again. The vertical distance between the offset diagonals also coincides with the periodicity of the system, which is around `t ≈ 4`.

On the right we see a structure typical of chaotic motion on a strange attractor such as the one of the Lorenz system: the orbit visits neighborhoods of previous points but then quickly diverges again. This results in many small diagonal lines.

## 5.D. Using the recurrence matrix

Plots are great, but most of the time one needs to go further. What information can be obtained from recurrence matrices? 


In [ ]:
RQA = rqa(R, theiler = 10, lmin = 5)

There are a lot of measures that one can compute from a recurrence matrix. Too many in fact, so for this notebook we will consider only a few of them: `determinism`, maximum and average diagonal line length and `divergence` (which is simply 1/(maximum diagonal length)).

In [ ]:
lor = Systems.lorenz()
for (i, ρ) in enumerate((69.75, 28.0))
    println("\nρ = $ρ ")
    set_parameter!(lor, 2, ρ)
    t, dt = 20.0, 0.01
    tr = trajectory(lor, t; dt = dt, Ttr = 200.0)
    
    ε = i == 1 ? 5.0 : 3.0
    R = RecurrenceMatrix(tr, ε)
    
    RQA = rqa(R, theiler = 10, lmin = 5)

    println("maximum diagonal: ", RQA.Lmax)
    println("average diagonal: ", RQA.L)
    println("determinism: ", RQA.DET)
    println("divergence (t-units): ", RQA.DIV/dt)
end

What can we understand from these numbers? 

1. For periodic systems the maximum and average diagonals are not really meaningful quantities. Their true values are `Inf`.
2. In the chaotic regime the average and maximum diagonal length both serve as an estimate of the *divergence* of nearby trajectories.
3. "Determinism" simply measures the portion of diagonal lines in the recurrence plot. It can be interpreted as the "predictability" of the trajectory, in the sense that approximately close states should have approximately similar future for small times.
4. "Divergence" is equal to 1/(maximum diagonal). It is just a measure for how fast do trajectories diverge. Again, for the periodic system we know that it theoretically should be zero.

### A final example: Laminarity

Besides these diagonal lines, there are other things that may show up in a recurrence plot, given a different trajectory. An example is vertical lines, which appear in e.g. intermittent systems.

For example, it is known that the logistic map is intermittent just before the period 3 window, so let's look at its recurrence plot there:

In [ ]:
logi = Systems.logistic(0.5; r = 3.828)

In the following I will use the **third iterate** of the logistic map (because the laminar phases are of period 3):

In [ ]:
x3 = trajectory(logi, 400; dt = 3) # <- every 3rd point with dt = 3

Rlogi = RecurrenceMatrix(x3, 0.1)
x, y = coordinates(Rlogi)

p1 = plot(x3, color=:black, lw=0.5, ylabel="x³")

p2 = scatter(x, y, markersize = 0.3, color=:black, xlabel="n", ylabel="n")
plot(p1, p2, layout=grid(2,1, heights=[0.3, 0.7]), leg=false, size=(500, 700))

`laminarity` is a function that counts the portion of the vertical lines in the recurrence plot:

In [ ]:
laminarity(Rlogi) # same as RQA(Rlogi).LAM

This number tells us how big of a portion of the trajectory is spend in a "laminar" phase. Equivalently how much portion of the trajectory is spend around a constant number +/- `ε`.